## Example 1 - Image Preparation
This notebook demonstrates how to prepare a thoracic x-ray computed tomography image for automatic segmentation.

In [1]:
# Import the necessary modules
import os
import SimpleITK as sitk
import numpy as np

#Visualisation, ImageProcessing
from robstools import ImageProcessing, Visualisation

# Matplotlib
%matplotlib
import matplotlib.pyplot as plt

# Widgets
import ipywidgets as widgets
from ipywidgets import interact, fixed

Using matplotlib backend: TkAgg


In [2]:
# Read in the images
idList = [i[-2:] for i in os.listdir("./Data") if "Case" in i]
imList = [sitk.ReadImage("./Data/Case_{0}/Case_{0}_CT.nii.gz".format(i)) for i in idList]

### Step 1. Automatic lung segmentation
Here we use an intensity-based algorithm to segment the lungs.

In [3]:
(boxList, maskList) = zip(*[(ImageProcessing.AutoLungSegment(im)) for im in imList])

#### Modifying the cropping box
Often the anterior limit of the box sits close to the heart border, so we extend it in this direction.

In [4]:
modifiedBoxList = []
for maskBox, im in zip(boxList, imList):
    sag0 = maskBox[0]
    cor0 = max([maskBox[1]-15, 0])
    ax0 =  max([maskBox[2]-5,0])
    sagD = maskBox[3]
    corD = min([maskBox[4]+15, im.GetSize()[1]-sag0])
    axD =  min([maskBox[5]+5,im.GetSize()[2]-ax0])

    cropBox = (sag0, cor0, ax0, sagD, corD, axD)
    modifiedBoxList.append(cropBox)

#### Plotting
Now we can plot the different images to make sure nothing went wrong

In [5]:
plt.ioff()
figList, axisList, cutList = zip(*[Visualisation.displaySlice(im, 'ortho') for im in imList]);

In [6]:
figList = [Visualisation.overlayContour(mask, fig, 'ortho', cut) for (mask, fig, cut) in zip(maskList, figList, cutList)]

/home/robbie/.local/lib/python2.7/site-packages/matplotlib/contour.py:1230: UserWarning: No contour levels were found within the data range.
  warnings.warn("No contour levels were found"


In [7]:
figList = [Visualisation.overlayBox(box, fig, 'ortho') for (box, fig) in zip(modifiedBoxList, figList)]

In [8]:
def figSelect(figureList, index):
    display( figureList[index] )

interact(figSelect, figureList=fixed(figList), index=range(len(figList)))

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUnaW5kZXgnLCBvcHRpb25zPSgwLCAxLCAyLCAzLCA0KSwgdmFsdWU9MCksIE91dHB1dCgpKSwgX2RvbV9jbGFzc2XigKY=


<function __main__.figSelect>

### Step 2. Cropping
Applying the same cropping box to the substructure images, and saving all of these new images.

In [36]:
croppedImList = [ImageProcessing.CropImage(im, box) for im, box in zip(imList, modifiedBoxList)]
[sitk.WriteImage(im, "./Data/Case_{0}/Case_{0}_CT_CROP.nii.gz".format(i)) for i, im in zip(idList, croppedImList)];

In [44]:
for i, box in zip(idList, modifiedBoxList):
    structureNameList = [s for s in os.listdir('./Data/Case_{0}'.format(i)) if ((('SProb' in s) or ('SBin' in s)) and ('CROP' not in s))]
    croppedImList = [ImageProcessing.CropImage(sitk.ReadImage('./Data/Case_{0}/{1}'.format(i,im)), box) for im in structureNameList]
       
    for cIm, s in zip(croppedImList, structureNameList):
        sName = s[6:][s[6:].find('_')+1:-7]
        sitk.WriteImage(cIm, './Data/Case_{0}/Case_{0}_{1}_CROP.nii.gz'.format(i, sName))
        